# LSTM Stock Predictor Using Closing Prices



## Data Preparation

The training and testing data will be prepared for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [24]:
import numpy as np
import pandas as pd
from path import Path

%matplotlib inline
import hvplot.pandas

In [25]:
# Set the random seed for reproducibility

from numpy.random import seed
seed(1)

from tensorflow import random
random.set_seed(2)

## Data Loading

In [27]:
# Load the FNG sentiment data for BTC
df_fng = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df_fng = df_fng.drop(columns="fng_classification")
df_fng.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [28]:
# Load BTC historic closing price
df_btc = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df_btc = df_btc.sort_index()
df_btc.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [31]:
# Join both DataFrames to a single one.
df = df_fng.join(df_btc, how="inner")
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


## Creating the Features X and Target y Data

The first step towards preparing the data is to create the input features vectors X and the target vector y. We will use the window_data() function to create these vectors.This function chunks the data up with a rolling window of X<sub>t</sub> - window to predict X<sub>t</sub>.The function returns two numpy arrays:

X: The input features vectors.

y: The target vector.

In [32]:

def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [34]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Try a window size anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[9114.719727 8870.820313 9251.269531 8218.049805 6937.080078 7701.25
  7592.720215 8260.69043  8696.830078 8569.290039]
 [8870.820313 9251.269531 8218.049805 6937.080078 7701.25     7592.720215
  8260.69043  8696.830078 8569.290039 8084.609863]
 [9251.269531 8218.049805 6937.080078 7701.25     7592.720215 8260.69043
  8696.830078 8569.290039 8084.609863 8911.269531]
 [8218.049805 6937.080078 7701.25     7592.720215 8260.69043  8696.830078
  8569.290039 8084.609863 8911.269531 8544.69043 ]
 [6937.080078 7701.25     7592.720215 8260.69043  8696.830078 8569.290039
  8084.609863 8911.269531 8544.69043  9485.639648]] 

y sample values:
[[ 8084.609863]
 [ 8911.269531]
 [ 8544.69043 ]
 [ 9485.639648]
 [10033.75    ]]


## Splitting Data Between Training and Testing Sets

In [35]:
# Use 70% of the data for training and the remaineder for testing
split = int(.7 * len(X))
X_train = X[:split - 1]
X_test = X[split:]
y_train = y[:split - 1]
y_test = y[split:]

##  Scaling Data with MinMaxScaler

In [37]:
# This uses MinMaxScaler to scale between 0 and 1. 
from sklearn.preprocessing import  MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

## Reshape Features Data for the LSTM Model

In [38]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.60761794]
  [0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]]

 [[0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]]

 [[0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]]

 [[0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]]

 [[0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]
  [0.64593307]]] 

X_test sample values:
[[[0.04675556]
  [0.03913222]
  [0.03975922]
  [0.03856719]
  [0.03679562]
  [0.03728835]
  [0.03974167]
  [0.04528668]
  [0.04528668]
  [0.07024855]]

 [[0.03913222]
  [0.03975922]
  [0.038567

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

We will:
1. Define the model architecture

2. Compile the model

3. Fit the model to the training data



## Importing the Keras Modules

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

##  Defining the LSTM RNN Model Structure

In [40]:
model = Sequential()

number_units = 10
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units, return_sequences=True,
    input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

##  Compiling the LSTM RNN Model

In [41]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [42]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 10)            480       
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 10)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 10)            840       
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 10)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 10)                840       
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

## Training the Model

In [43]:
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Train on 371 samples
Epoch 1/10
371/371 [==============================] - 10s 27ms/sample - loss: 0.0321
Epoch 2/10
371/371 [==============================] - 4s 10ms/sample - loss: 0.0227
Epoch 3/10
371/371 [==============================] - 4s 9ms/sample - loss: 0.0226
Epoch 4/10
371/371 [==============================] - 4s 10ms/sample - loss: 0.0193
Epoch 5/10
371/371 [==============================] - 4s 10ms/sample - loss: 0.0157
Epoch 6/10
371/371 [==============================] - 4s 10ms/sample - loss: 0.0151
Epoch 7/10
371/371 [==============================] - 3s 9ms/sample - loss: 0.0144
Epoch 8/10
371/371 [==============================] - 4s 10ms/sample - loss: 0.0130
Epoch 9/10
371/371 [==============================] - 4s 10ms/sample - loss: 0.0123
Epoch 10/10
371/371 [==============================] - 4s 10ms/sample - loss: 0.0123


---

## Model Performance

In this section, you will evaluate the model using the test data. 

We will:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

## Evaluate the Model

In [44]:
model.evaluate(X_test, y_test)

160/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.05162762689869851

##  Making Predictions

In [45]:
predicted = model.predict(X_test)

In [46]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

## Plotting Predicted Vs. Real Prices

In [47]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()

,Real,Predicted
0,3924.239990,3713.654297
1,3974.050049,3725.932129
2,3937.040039,3747.568604
3,3983.530029,3772.726562
4,4149.089844,3799.946045


In [48]:
# Plot the real vs predicted values as a line chart
stocks.head(100).hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)